In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm


%matplotlib inline

In [2]:
df = None
# load all the datasets and concatenate them in a single dataframe
# during the process the run ids are renamed s.t. no two different runs have the same id in the final dataframe
id_offset = 0
for i in range(1,6):
    df_tmp = pd.read_csv(f'dataset_{i}.csv')
    current_ids = list(df_tmp['run_id'].unique())
    df_tmp['run_id'] = df_tmp['run_id'].apply(lambda x: id_offset + current_ids.index(x))
    df = pd.concat([df,df_tmp])
    id_offset += len(current_ids)
    
# drop duplicated indexes by resetting to default ones
df = df.reset_index(drop=True)
assert df.index.has_duplicates == False

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_1.csv'

## Data cleaning and filtering

In [ ]:
# remove data collected before wednesday 06-04-2022 which is the day we started to collect real data
df = df[df['recorded_at'] > '2022-04-06']

In [ ]:
# select only needed columns
df_new = df[['run_id', 'trial_index','response']]
# remove welcome and final trials, those are 0 and 19 trials respectively
df_final = df_new[(df_new['trial_index'] != 0) & (df_new['trial_index'] != 19)]

In [ ]:
df_final.head()

In [ ]:
# get the size of the datatset based on amount of runs
all_ids = df_final.groupby('run_id')
all_size = len(all_ids)
all_size

In [ ]:
# get the size invalid runs if the number of answered quistions is less than 18
df_incomplete = df_final.groupby('run_id').filter(lambda x: x.shape[0] < 18)
invalid_size = len(df_incomplete.groupby('run_id'))
invalid_size

In [ ]:
# get the size valid runs if the number of answered quistions is equal to 18
df_valid = df_final.groupby('run_id').filter(lambda x: x.shape[0] == 18)
valid_size = len(df_valid.groupby('run_id'))
valid_size

### filter people based on control answer 

In [ ]:
control_yes = df_valid.loc[df_valid['trial_index']==17,'response'].apply(lambda x: 'Yes' in json.loads(x).values())
invalid_ids = df_valid.loc[control_yes.index,:][~control_yes].run_id.to_list()

In [ ]:
# remove samples from partecipants who failed the boolean control question
df_valid = df_valid[~df_valid.run_id.isin(invalid_ids)]

In [ ]:
control_eyes = df_valid.loc[df_valid['trial_index']==18,'response'].str.contains('.*[eE]yes{0,1}.*|.*[yY]eux.*|.*œil.*')
# look up the text answers on the control question.
df_valid.loc[control_eyes.index,:][~control_eyes]

In [ ]:
# remove the answers that suggests the partecipant didn't notice the difference in eyes size
invalid_ids = [15, 18, 112, 116, 135]

df_valid = df_valid[~df_valid.run_id.isin(invalid_ids)].astype({'run_id':int, 'trial_index':int})

### extract demographic questions

In [ ]:
# select only demographics questions (question numbr is 16)
valid_demographics = df_valid.loc[df_valid['trial_index'] == 16,:]

In [ ]:
# convert values in json format to columns
valid_demographics = valid_demographics.assign(response_json=lambda x: x["response"].apply(lambda x: [json.loads(x)]))
valid_response = pd.json_normalize(valid_demographics.to_dict(orient="records"),meta=["run_id","trial_index"], record_path=["response_json"])
columns = valid_response.columns.to_list()
# reorder columns, put run_id and trial_index in front
columns.remove("trial_index")
columns.remove("run_id")
columns.insert(0,"run_id")
columns.insert(1,"trial_index")
valid_response = valid_response[columns]
valid_response

### extract data about acceptance:
for each answer extract:
- which image was displayed (0-14)
- which was the condition of the image w.r.t eyes size (0 = small, 1 = normal, 2 = big)
- what did the partecipant answer (0-5)

In [ ]:
# create a dataframe containing the acceptance responses
df_acceptance = df_valid.loc[df_valid['trial_index'] < 16,:]

df_acceptance

In [ ]:
# extract values from the response json and add them as columns of the dataframe

# extract images info i.e. image index(0-14) and condition index(0 = small, 1 = normal, 2 = big)
tmp_img_info = df_acceptance.response \
                            .apply(lambda x: json.loads(x)) \
                            .apply(lambda x: list(x.keys())[0]).str.extract(pat='image_([0-9]|[1][0-4])_([0-2])')

# extract partecipant responses (0-5)
df_acceptance = df_acceptance.assign(image_index = tmp_img_info[0].astype(int), condition_index = tmp_img_info[1].astype(int))
df_acceptance.loc[:,'response'] = df_acceptance.response \
                            .apply(lambda x: json.loads(x)) \
                            .apply(lambda x: list(x.values())[0]) \
                            .apply(int)
df_acceptance

## Data Analysis

In [ ]:
df_runs = valid_response

In [ ]:
assert len(df_runs) == len(df_acceptance.groupby(by='run_id').count())
assert len(df_runs)*15 == len(df_acceptance)

print(f"number of runs (total number of partecipants): {len(df_runs)}")
print(f"number of datapoints about acceptance: {len(df_acceptance)}( = 15 * {len(df_runs)})")

In [ ]:
df_runs.gender.value_counts().plot(kind='bar')
plt.title("gender distribution")
plt.xlabel("gender")
plt.ylabel("# of partecipants")
plt.show()
# TODO: decide whether to keep bar chart or pie chart for gender distribution

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(15,15))
ax[0,0].pie(x = df_runs.gender.value_counts().sort_index(), labels = sorted(df_runs.gender.unique()), autopct='%1.0f%%')
ax[1,0].pie(x = df_runs.level_educational_background.value_counts().sort_index(), labels = sorted(df_runs.level_educational_background.unique()), autopct='%1.0f%%')
ax[0,1].pie(x = df_runs.field_educational_background.value_counts().sort_index(), labels = sorted(df_runs.field_educational_background.unique()), autopct='%1.0f%%')
ax[1,1].pie(x = df_runs.nature_time.value_counts().sort_index(), labels = sorted(df_runs.nature_time.unique()), autopct='%1.0f%%')

plt.show()
# TODO: add titles etc...

TODO: decide whether to keep either the pie chart or the bar chart for categorical variables' distributions

In [ ]:
# extract age as numeric values from the text responses
df_runs['age'] = df_runs['age'].str.extract('([1-9]{0,2}[0-9])').astype(int)

In [ ]:
print(f"the mean age is: {df_runs.age.mean()}")
print(f"the std of the age is: {df_runs.age.std()}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))

df_runs.age.hist(bins=20, ax=ax1)
ax1.set_title("age distribution")
ax1.set_xlabel("age")
ax1.set_ylabel("# of participants")

num_of_females = df_runs.age[df_runs.gender=='Female'].count()
num_of_males = df_runs.age[df_runs.gender=='Male'].count()

female_ages_relfreq = (df_runs.age[df_runs.gender=='Female'].value_counts() / num_of_females).sort_index()
male_ages_relfreq = (df_runs.age[df_runs.gender=='Male'].value_counts() / num_of_males).sort_index()

pd.concat([female_ages_relfreq.rename('female'), male_ages_relfreq.rename('male')], axis=1).fillna(0).plot.bar(ax=ax2)
ax2.set_title("age distribution of female and males")
ax2.set_xlabel('age')
ax2.set_ylabel('relative frequencies')

plt.show()

in the above right-most plot, the relatives frequences are calculated whitin the group(male or female) and not just on the total number of partecipants. In this way it is possible to compare the two distributions even thought the groups are very unbalanced (females are two times more than the males)

In [ ]:
# plt.pie(x = df_runs.level_educational_background.value_counts().sort_index(), labels = sorted(df_runs.level_educational_background.unique()), autopct='%1.0f%%')
# plt.title('highest level of education distribution')
# plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))

df_runs.level_educational_background.hist(bins=20, ax=ax1)
ax1.set_title("highest level of education distribution")
# ax1.set_xlabel("highest level of education")
ax1.set_ylabel("# of partecipants")
ax1.tick_params(axis='x', rotation=45)

female_edulevel_relfreq = (df_runs.level_educational_background[df_runs.gender=='Female'].value_counts() / num_of_females).sort_index()
male_edulevel_relfreq = (df_runs.level_educational_background[df_runs.gender=='Male'].value_counts() / num_of_males).sort_index()

pd.concat([female_edulevel_relfreq.rename('female'), male_edulevel_relfreq.rename('male')], axis=1).fillna(0).plot.bar(ax=ax2)
ax2.set_title("highest level of education distribution of female and males")
# ax2.set_xlabel('highest level of education')
ax2.set_ylabel('relative frequencies')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))

df_runs.field_educational_background.hist(bins=20, ax=ax1)
ax1.set_title("field of education distribution")
# ax1.set_xlabel("field of education")
ax1.set_ylabel("# of partecipants")
ax1.tick_params(axis='x', rotation=90)

female_edulevel_relfreq = (df_runs.field_educational_background[df_runs.gender=='Female'].value_counts() / num_of_females).sort_index()
male_edulevel_relfreq = (df_runs.field_educational_background[df_runs.gender=='Male'].value_counts() / num_of_males).sort_index()

pd.concat([female_edulevel_relfreq.rename('female'), male_edulevel_relfreq.rename('male')], axis=1).fillna(0).plot.bar(ax=ax2)
ax2.set_title("field of education distribution of female and males")
# ax2.set_xlabel('field of education')
ax2.set_ylabel('relative frequencies')

plt.show()

In [ ]:
# plt.pie(x = df_runs.field_educational_background.value_counts().sort_index(), labels = sorted(df_runs.field_educational_background.unique()), autopct='%1.0f%%')
# plt.title('field of education distribution')
# plt.show()

In [ ]:
df_small_acceptance = df_acceptance[df_acceptance.condition_index==0]
df_normal_acceptance = df_acceptance[df_acceptance.condition_index==1]
df_big_acceptance = df_acceptance[df_acceptance.condition_index==2]

# assert that the number of small, normal and big images has are the same over the collected responses
assert len(df_small_acceptance) == len(df_normal_acceptance) == len(df_big_acceptance)

print(f"total number of responses: {len(df_small_acceptance)+len(df_normal_acceptance)+len(df_big_acceptance)}:")
print(f"small eyes responses: \t {len(df_small_acceptance)}")
print(f"normal eyes responses: \t {len(df_normal_acceptance)}")
print(f"big eyes responses: \t {len(df_big_acceptance)}\n")

print("the average degree of acceptance w.r.t the eyes size is (mean ± std):")
print(f"condition 0 - small eyes: \t {df_small_acceptance.response.mean():.2f} ± {df_small_acceptance.response.std():.2f}")
print(f"condition 1 - normal eyes: \t {df_normal_acceptance.response.mean():.2f} ± {df_normal_acceptance.response.std():.2f}")
print(f"condition 2 - big eyes: \t {df_big_acceptance.response.mean():.2f} ± {df_big_acceptance.response.std():.2f}")





In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))

df_acceptance['response'].hist(bins=6, ax= ax1)

ax1.set_title('distribution of the degree of acceptance')
ax1.set_xlabel('degree of acceptance')
ax1.set_ylabel('# of responses')


num_of_small = df_acceptance.response[df_acceptance.condition_index==0].count()
num_of_normal = df_acceptance.response[df_acceptance.condition_index==1].count()
num_of_big = df_acceptance.response[df_acceptance.condition_index==2].count()

small_acceptance_relfreq = (df_acceptance.response[df_acceptance.condition_index==0].value_counts() / num_of_small).sort_index()
normal_acceptance_relfreq = (df_acceptance.response[df_acceptance.condition_index==1].value_counts() / num_of_normal).sort_index()
big_acceptance_relfreq = (df_acceptance.response[df_acceptance.condition_index==2].value_counts() / num_of_big).sort_index()

pd.concat([\
        small_acceptance_relfreq.rename('smaller eyes'),\
        normal_acceptance_relfreq.rename('normal eyes'),\
        big_acceptance_relfreq.rename('bigger eyes')\
    ],axis=1).fillna(0).plot.bar(ax=ax2)


ax2.set_title('distribution of the degree of acceptance w.r.t. of eyes size')
ax2.set_xlabel('degree of acceptance')
ax2.set_ylabel('relative frequencies')

plt.show()


In [ ]:
stats.ttest_ind(df_small_acceptance.response, df_normal_acceptance.response)

In [ ]:
stats.ttest_ind(df_big_acceptance.response, df_normal_acceptance.response , alternative='less')

In [ ]:
stats.ttest_ind(df_big_acceptance.response, df_small_acceptance.response, alternative='less')

### observation

The t-test does not indicate a significant difference between the mean degree of acceptance w.r.t small and normal eyes respectively. However, with p-value=0.006 it suggests that the mean degree of acceptance w.r.t. big eyes is significant greater than the mean degree w.r.t to small eyes. Moreover, it suggests that the mean degree of acceptance w.r.t. big eyes is significant greater than the mean degree w.r.t to normal eyes (p-value=0.03)

In [ ]:
stats.spearmanr(df_acceptance.condition_index, df_acceptance.response)

In [ ]:
stats.pearsonr(df_acceptance.condition_index, df_acceptance.response)

### observation
Even the correlation tests support (p-values < 0.02) our hypothesis suggesting that there is a slightly negative correlation between degree of acceptance and condition index i.e. increasing the eyes size, the degree of acceptance decreases.

In particular, the pearson coefficent suggests a linear correlation between the two variables. We should therefore proceed with a regression analysis to understand if additional correlations are present which might indicates the presence of cofounding variables

### Regression analysis

In [ ]:
df_regression = df_runs.merge(df_acceptance, left_on='run_id', right_on='run_id')

In [ ]:
df_regression

In [ ]:
import plotly.express as px

df = df_regression[df_regression['gender'] != 'Other']

fig = px.box(df, x="condition_index", y="response", color="gender") #field_educational_background
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [ ]:
#We need to separate the 'feel_rat' column in different columns, depending on what the subjects answered
#Here we check what is the maximum number of answers for the question to know the number of columns to use
max(np.array([len(df_regression['feel_rat'][i]) for i in range(1605)]))

In [ ]:
df_regression[['feel_rat1', 'feel_rat2', 'feel_rat3']] = df_regression['feel_rat'].apply(str).str.strip('[]').str.split(',\s*', expand = True)
df_regression


In [ ]:
pd.unique(df_regression['feel_rat1'])

In [ ]:
# TODO: extract 'feel_rat' from the list and add dummy variables for feel rat and other categorical variable.
# TODO:Also convert nature time to numerical
df_dummies = pd.get_dummies(df_regression[['gender', 'response', 'condition_index', 'level_educational_background','field_educational_background', 'dieatary_habit', 'feel_rat1', 'feel_rat2', 'feel_rat3']])
df_dummies["feel_rat_'Indifference'"] = df_dummies["feel_rat1_'Indifference'"] + df_dummies["feel_rat2_'Indifference'"] + df_dummies["feel_rat3_'Indifference'"]
df_dummies["feel_rat_'Disgust'"] = df_dummies["feel_rat1_'Disgust'"] + df_dummies["feel_rat2_'Disgust'"] + df_dummies["feel_rat3_'Disgust'"]
df_dummies["feel_rat_'Fear'"] = df_dummies["feel_rat1_'Fear'"] + df_dummies["feel_rat2_'Fear'"] #+ df_dummies["feel_rat3_'Fear'"]
df_dummies["feel_rat_'Sympathy'"] = df_dummies["feel_rat1_'Sympathy'"] + df_dummies["feel_rat2_'Sympathy'"] #+ df_dummies["feel_rat3_'Sympathy'"]
df_dummies["feel_rat_'Other'"] = df_dummies["feel_rat1_'Other'"] + df_dummies["feel_rat2_'Other'"] #+ df_dummies["feel_rat3_'Other'"]



In [ ]:
drop_list = ['rat1', 'rat2', 'rat3']
for i in range(len(drop_list)):
    df_dummies = df_dummies.drop(df_dummies.filter(like=drop_list[i]).columns, 1)
df_dummies

In [ ]:
df_dummies.to_numpy()

In [ ]:
df_regression.columns



In [ ]:
#For the model we will dismiss the "other" answers as not enough data is given (not enough participants replied that)
df_regression = df_regression[(df_regression['gender'] != 'Other') &(df_regression['feel_rat1'] != "'Other'")
                             & (df_regression['dieatary_habit'] != 'Other')
                             & (df_regression['field_educational_background'] != 'Other')]

In [ ]:
model = sm.ols(formula="response ~ condition_index + C(gender) + C(feel_rat1) + C( dieatary_habit) + C(field_educational_background )", 
                data=df_regression)
#C( dieatary_habit + field_educational_background) + C(gender)
#gender + dieatary_habit + field_educational_background

res = model.fit()

print(res.summary())

When adding to the regression analysis the feel_rat2 or feel_rat3 variables the R2 increases significantly, but it is because the number of observations is reduced (most of the participants only put one feeling for rats and no more). Maybe we should have restricted the field to the stronger feeling. 

In [ ]:
# feature names
variables = res.params.index

# quantifying uncertainty!

# coefficients
coefficients = res.params.values

# p-values
p_values = res.pvalues

# standard errors
standard_errors = res.bse.values

#confidence intervals
res.conf_int()

In [ ]:
#sort them all by coefficients
l1, l2, l3, l4 = zip(*sorted(zip(coefficients[1:], variables[1:], standard_errors[1:], p_values[1:])))

# in this case, we index starting from the first element, not to plot the intercept

# we will use standard errors, instead of CIs
# two standard errors approximate the CIs (you can actually see in the summary table that
# +/2 SI is equivalent to the CIs)

l2

In [ ]:
#fancy plotting

plt.errorbar(l1, np.array(range(len(l1))), xerr= 2*np.array(l3), linewidth = 1,
             linestyle = 'none',marker = 'o',markersize= 3,
             markerfacecolor = 'black',markeredgecolor = 'black', capsize= 5)

plt.vlines(0,0, len(l1), linestyle = '--')

plt.yticks(range(len(l2)),l2);

From the obtained results, it seems that the participant characteristics which correlate negatively the most the results is a vegan/vegetarian diet and sympathy for rats. On the other hand, being a male, studying in the healthcare or social sciences field or fearing rats seem to create a tendency for accepting the animal experimentation more. The eye size has a low correlation with the results, nonetheless considering the standard error and confidence interval, the correlation is always negative. This means the effect of the eye size is consistent.

###  We will try now to perform the analysis only with the demographic answers we consider give us more information from the dummy table

In [ ]:
df_dummies.rename(columns = {'field_educational_background_Engineering, IT, Applied Science':'field_educational_background_Engineering', 
                     'field_educational_background_Humanities and Social Science':'field_educational_background_Humanities',
                             "field_educational_background_Life Sciences, Medicine, Healthcare":"field_educational_background_Healthcare",
                     'dieatary_habit_No specific diet':'dieatary_habit_No_specific_diet', 
                     "feel_rat_'Indifference'":'feel_rat_Indifference' ,
                     "feel_rat_'Disgust'":'feel_rat_Disgust',
                     "feel_rat_'Fear'": 'feel_rat_Fear', 
                     "feel_rat_'Sympathy'":'feel_rat_Sympathy'}, inplace = True)

In [ ]:
df_dummies.columns

In [ ]:
model2 = sm.ols(formula="response ~ condition_index *(field_educational_background_Engineering + field_educational_background_Humanities + field_educational_background_Healthcare) * (dieatary_habit_No_specific_diet + dieatary_habit_Vegan + dieatary_habit_Vegetarian) * (feel_rat_Fear + feel_rat_Disgust + feel_rat_Sympathy+feel_rat_Indifference)", 
                data=df_dummies)
#C( dieatary_habit + field_educational_background) + C(gender)
#gender + dieatary_habit + field_educational_background
#+ feel_rat_'Disgust' 
res2 = model2.fit()

print(res2.summary())

In [ ]:
# feature names
variables = res2.params.index

# quantifying uncertainty!

# coefficients
coefficients = res2.params.values

# p-values
p_values = res2.pvalues

# standard errors
standard_errors = res2.bse.values

#confidence intervals
res2.conf_int()

In [ ]:
#sort them all by coefficients
l1, l2, l3, l4 = zip(*sorted(zip(coefficients[1:], variables[1:], standard_errors[1:], p_values[1:])))

# in this case, we index starting from the first element, not to plot the intercept

# we will use standard errors, instead of CIs
# two standard errors approximate the CIs (you can actually see in the summary table that
# +/2 SI is equivalent to the CIs)


In [ ]:
new_l1 = []
new_l2 = []
new_l3 = []

for i in range(len(l1)):
    if l1[i]<-1.5 or l1[i]>2:
        new_l1.append(l1[i])
        new_l2.append(l2[i])
        new_l3.append(l3[i])
    

In [ ]:
#fancy plotting

plt.errorbar(new_l1, np.array(range(len(new_l1))), xerr= 2*np.array(new_l3), linewidth = 1,
             linestyle = 'none',marker = 'o',markersize= 3,
             markerfacecolor = 'black',markeredgecolor = 'black', capsize= 5)

plt.vlines(0,0, len(new_l1), linestyle = '--')

plt.yticks(range(len(new_l2)),new_l2);

On the above plot we can see the combination of variables which explain most of the variance in the results. 

### TODO: ANOVA

In [ ]:
df_pre_anova = df_acceptance[['condition_index', 'response']]
df_pre_anova = df_pre_anova.pivot(index = None, columns=['condition_index'], values=['response'])

df_anova = pd.DataFrame()
df_anova['Condition0']= df_pre_anova[('response', 0)].dropna().reset_index(drop = True)
df_anova['Condition1'] = df_pre_anova[('response', 1)].dropna().reset_index(drop=True)
df_anova['Condition2'] = df_pre_anova[('response', 2)].dropna().reset_index(drop=True)
df_anova

In [ ]:
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(df_anova['Condition0'], df_anova['Condition1'], df_anova['Condition2'])
print('The results for one-way ANOVA test are: fvalue = ' + str(round(fvalue,3)) + ' and pvalue = ' + str(round(pvalue,3)))


In [ ]:
model = sm.ols(formula="response ~ condition_index", data=df_regression).fit()
print(model.summary())

In [ ]:
# get ANOVA table as R like output
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey = pairwise_tukeyhsd(endog=df_regression['response'], groups=df_regression['condition_index'], alpha=0.05)
print(tukey)